# Painting reconciliation

From all our illustrations, we want to find metadata about them. What is the name of the author ? The title of the artwork ? Its dimensions ? 

For that, we will query Smartify, an API which recognizes artworks.

In [2]:
from selenium.webdriver.common.by import By
import pyautogui
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import os
import time
import splitfolders
import glob
import random
import shutil
from tqdm import tqdm
import pandas as pd

We divide our illustrations in 10 batches, just so that it is easier to process.

In [3]:
N_BATCHES = 10

def divide_in_batches():
    all_illus = glob.glob("illustrations/*.jpg")
    batch_len = int(len(all_illus)/N_BATCHES)
    random.seed(0)
    random.shuffle(all_illus)
    
    for i in tqdm(range(N_BATCHES)):
        for illu in all_illus[i*batch_len:(i+1)*batch_len]:
            dest = './batch' + str(i) + '/'
            shutil.copy(illu, dest)

divide_in_batches()

In [16]:
# a function to retrieve - from the page - the painting information
def get_painting_info(driver, illu_id):
    title = driver.find_element(by=By.XPATH, value="/html/body/div/div/div/div/div/div/div[4]/div/div/h1").get_attribute("innerHTML")
    specs = driver.find_element(by=By.XPATH, value="/html/body/div/div/div/div/div/div/div[4]/div/div/p").get_attribute("innerHTML")
    author = driver.find_element(by=By.XPATH, value="/html/body/div/div/div/div/div/div/div[4]/a/div/h3").get_attribute("innerHTML")
    try:
        link = driver.find_element(by=By.XPATH, value="/html/body/div/div/div/div/div/div/div[4]/div[3]/a").get_attribute("href")
    except:
        link = ''
    return dict({'id': illu_id, 'title': title, 'specs': specs, 'author': author, 'link': link})

In [145]:
# All the work of querying and scraping the website is done here
def retrieve_for_batch(batch):
    driver = webdriver.Chrome('./chromedriver.exe')
    driver.get("https://smartify.org/")
    time.sleep(1)
    
    all_illus = glob.glob(batch + "/*.jpg")
    paths = ['documents', 'DFKV', 'DFKV-illustrations', '5_illustration_enrichment', batch, all_illus[0].split('\\')[-1].split('.')[0]]
    data = []
    # first illustration
    previous_url = driver.current_url
    button = driver.find_element(by=By.XPATH, value="/html/body/div/div/div/div/div/header/div/section/div[2]/div[1]/div/div/button")
    button.click() # This opens the windows file selector
    time.sleep(1)
    
    for step in paths:
        time.sleep(1)
        pyautogui.write(step)
        pyautogui.press('enter')  
    time.sleep(1)
    
    if previous_url != driver.current_url:
        data.append(get_painting_info(illu_name))
            
    for illu in tqdm(all_illus[1:]):

        previous_url = driver.current_url

        illu_name = illu.split('\\')[-1].split('.')[0]
        button = driver.find_element(by=By.XPATH, value="/html/body/div/div/div/div/div/header/div/section/div[2]/div[1]/div/div/button")
        button.click() # This opens the windows file selector
        time.sleep(1)
        pyautogui.write(illu_name) 
        pyautogui.press('enter')
        time.sleep(2)
        if previous_url != driver.current_url:
            data.append(get_painting_info(driver, illu_name))
            df = pd.DataFrame(data)
            df.to_csv(batch + '.csv') # intermediate saves

    df = pd.DataFrame(data)
    df.to_csv(batch + '.csv')
    return df

In [146]:
# We do that for all the batches
df = retrieve_for_batch('batch9')

C:\Users\emichelet\AppData\Local\Temp\ipykernel_13168\735552752.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver.exe')
100%|██████████| 454/454 [24:56<00:00,  3.30s/it]
